In [109]:
import streamlit as st
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pytz
import datetime as datetime
import os
import sys
import numpy as np
import plotly.express as px


In [110]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'FPL')))


In [111]:
from FPL.fpl_api_collection import (
    get_bootstrap_data,
    get_current_gw,
    get_fixt_dfs,
    get_league_table
)
from FPL.fpl_utils import (
    define_sidebar,
    get_annot_size,
    map_float_to_color,
    get_text_color_from_hash,
    get_rotation
)
from FPL.fpl_params import (
    TIMEZONES_BY_CONTINENT,
    AUTHOR_CONTINENT,
    AUTHOR_CITY
)

In [112]:
team_fdr_df, team_fixt_df, team_ga_df, team_gf_df = get_fixt_dfs()

events_df = pd.DataFrame(get_bootstrap_data()['events'])

gw_min = min(events_df['id'])
gw_max = max(events_df['id'])

ct_gw = get_current_gw()

In [113]:
sui=team_fixt_df.reset_index()


val=team_fdr_df.reset_index()

In [114]:
# Rename the first column to 'Team'
sui.rename(columns={0: 'Team'}, inplace=True)
val.rename(columns={0: 'Team'}, inplace=True)

# Create new column names, keeping 'Team' as the first column
sui.columns = ['Team'] + [f'GW{col}' for col in range(1, len(sui.columns))]
val.columns = ['Team'] + [f'GW{col}' for col in range(1, len(val.columns))]

In [115]:
sui
val

,Team,GW1,GW2,GW3,GW4,GW5,GW6,GW7,GW8,GW9,...,GW29,GW30,GW31,GW32,GW33,GW34,GW35,GW36,GW37,GW38
0,ARS,2.0,4.0,2.0,4.0,5.0,2.0,2.0,3.0,5.0,...,4.0,2.0,2.0,2.0,2.0,2.0,2.0,5.0,3.0,2.0
1,AVL,2.0,5.0,2.0,2.0,2.0,2.0,3.0,3.0,2.0,...,5.0,3.0,3.0,2.0,3.0,5.0,2.0,3.0,4.0,3.0
2,BOU,2.0,3.0,2.0,4.0,5.0,2.0,2.0,5.0,4.0,...,2.0,2.0,2.0,2.0,2.0,3.0,5.0,3.0,5.0,2.0
3,BRE,2.0,5.0,2.0,5.0,4.0,2.0,2.0,3.0,2.0,...,3.0,4.0,4.0,5.0,2.0,2.0,3.0,2.0,2.0,2.0
4,BHA,2.0,3.0,5.0,2.0,3.0,4.0,4.0,4.0,2.0,...,5.0,3.0,2.0,2.0,3.0,2.0,3.0,2.0,5.0,4.0
5,CHE,5.0,2.0,2.0,3.0,2.0,2.0,3.0,5.0,3.0,...,5.0,4.0,3.0,2.0,3.0,2.0,5.0,4.0,3.0,2.0
6,CRY,3.0,2.0,4.0,2.0,3.0,2.0,5.0,2.0,4.0,...,4.0,2.0,2.0,5.0,2.0,5.0,3.0,4.0,2.0,5.0
7,EVE,2.0,4.0,2.0,4.0,2.0,2.0,3.0,2.0,2.0,...,2.0,5.0,5.0,2.0,5.0,4.0,2.0,3.0,2.0,4.0
8,FUL,3.0,2.0,2.0,2.0,3.0,2.0,5.0,3.0,2.0,...,4.0,5.0,5.0,3.0,4.0,2.0,4.0,2.0,3.0,5.0
9,IPS,5.0,5.0,2.0,3.0,2.0,3.0,2.0,2.0,3.0,...,3.0,3.0,2.0,4.0,5.0,4.0,2.0,2.0,2.0,2.0


In [117]:
# Combine teams from both DataFrames
teams = pd.concat([sui['Team'], val['Team']]).unique()

# Initialize the FDR matrix
fdr_matrix = pd.DataFrame(index=teams, columns=['GW1', 'GW2'])

# Populate the FDR matrix based on GW matches
for team in teams:
    # Calculate FDR values based on team matchups
    gw1_fdr = np.random.randint(1, 6)  # Example random score for GW1
    gw2_fdr = np.random.randint(1, 6)  # Example random score for GW2
    fdr_matrix.loc[team] = [gw1_fdr, gw2_fdr]

# Convert FDR matrix to a format suitable for styling
fdr_matrix = fdr_matrix.reset_index()
fdr_matrix = fdr_matrix.melt(id_vars='index', var_name='GameWeek', value_name='FDR')
fdr_matrix.rename(columns={'index': 'Team'}, inplace=True)

# Define a coloring function based on the FDR values
def color_fdr(team, game_week):
    value = fdr_matrix[(fdr_matrix['Team'] == team) & (fdr_matrix['GameWeek'] == game_week)]['FDR'].values[0]
    if value <= 2:
        return 'background-color: red'  # High difficulty
    elif value <= 4:
        return 'background-color: yellow'  # Medium difficulty
    else:
        return 'background-color: green'  # Low difficulty

# Create a filtered FDR matrix for styling
filtered_fdr_matrix = fdr_matrix.pivot(index='Team', columns='GameWeek', values='FDR')

# Apply the styling to the filtered FDR matrix
styled_filtered_fdr_table = filtered_fdr_matrix.style.apply(
    lambda row: [color_fdr(row.name, col) for col in row.index], axis=1
)

# Streamlit app to display the styled table
st.title("Fixture Difficulty Rating (FDR) Matrix")
st.write(styled_filtered_fdr_table)

2024-10-17 15:42:54.166 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 15:42:54.166 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 15:42:54.167 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 15:42:54.637 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 15:42:54.638 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
